## Формирование csv-файлов по данным из 1С

In [ ]:
'''
Скрипт для распределения выгруженных из 1С ОСВ по папкам в соответствии с периодом, по которому сформированы ОСВ
'''
# импортируем библиотеки
from tqdm import tqdm
import pandas as pd
import os
import re
import shutil
import numpy as np
import sqlalchemy as db
import psycopg2

In [ ]:
# запишем путь к папке с исходными данными в переменную
source = os.path.join('H:',
                      'Исходные данные')
# создадим список с названиями всех файлов в папке с исходными данными с расширением .xlsx
raw_data_name_list = [raw_data for raw_data in os.listdir(source) if raw_data.startswith('текущий день_ОСВ') and
                      raw_data.endswith('.xlsx') or raw_data.startswith('предыдущий месяц_ОСВ') and
                      raw_data.endswith('.xlsx')]
raw_data_name_list

In [ ]:
print('Выполнение скрипта "osv_distribution_by_folders" (распределение ОСВ по папкам)')
print(f'Чтение ОСВ из "{source}"')
# создадим справочник из всех датафреймов в списке 'raw_data_name_list'
all_raw_data = {raw_data:pd.read_excel(os.path.join(source, f'{raw_data}')) for raw_data in tqdm(raw_data_name_list)}

In [ ]:
# создадим список из номеров строк, в которых встречается слово 'Период' в excel файле
period_location_row = [raw_data.apply(lambda row: row.astype(str)
                                      .str.contains('Период').any(), axis=1)
                               .loc[lambda x: x==True].index[0] for raw_data in all_raw_data.values()]

In [ ]:
# создадим список из названий столбцов, в которых встречается слово 'Период' в excel файле
period_location_column = [raw_data.apply(lambda row: row.astype(str)
                                                        .str
                                                        .contains('Период')
                                                        .any(), axis=0)
                                  .loc[lambda x: x==True].index[0] for raw_data in all_raw_data.values()]

In [ ]:
# объединим списки 'period_location_row' и 'period_location_column'
# в список кортежей с номером строки и названием столбца
period_location = list(zip(period_location_row, period_location_column))

In [ ]:
# создадим пустые списки
period_list_start = []
period_list_end = []

# создадим счётчик
x = 0
# при помощи цикла заполним список 'period_list_start' значениями начала периода у каждого файла
for raw_data in all_raw_data.values():
    period_list_start.append(raw_data.loc[period_location[x][0], period_location[x][1]][8:18])
    x += 1

# создадим счётчик
x = 0
# при помощи цикла заполним список 'period_list_end' значениями конца периода у каждого файла
for raw_data in all_raw_data.values():
    period_list_end.append(raw_data.loc[period_location[x][0], period_location[x][1]][-10:])
    x += 1
    
# поменяем местами год, месяц и день, и запишем в новую переменную 'period_list_end_reversed'
period_list_end_reversed = [period[len(period) - 4:] + period[2:len(period) - 4] + period[:2] for period in period_list_end]

In [ ]:
# создадим список из номеров строк, в которых встречается слово 'Счет' в excel файле в столбце 'Unnamed: 0',
# чтобы в дальнейшем оставить только строки, содержащие таблицу с данными
rows_to_delete_index = []
for raw_data in all_raw_data.values():
    rows_to_delete_index.append(list(raw_data['Unnamed: 0']).index('Счет'))

In [ ]:
# объединим списки 'raw_data_name_list' и 'rows_to_delete_index'
# в список кортежей с названием файла и номером строки для удаления
rows_to_delete = list(zip(raw_data_name_list, rows_to_delete_index))

In [ ]:
# при помощи цикла обновим словарь 'all_raw_data', удалив лишние строки из каждого датафрейма
for raw_data in all_raw_data.items():
    for row in rows_to_delete:
        if raw_data[0] == row[0]:
            all_raw_data.update({raw_data[0]: raw_data[1][row[1]+1:]})

In [ ]:
# при помощи цикла удалим пустые столбцы в каждом датафрейме из словаря 'all_raw_data'
all_raw_data = {raw_data[0]: raw_data[1].dropna(how='all', axis=1) for raw_data in all_raw_data.items()}

In [ ]:
# при помощи цикла заменим текущие названия столбцов на значения из 1-ой строки
# и удалим 2 лишние строки в каждом датафрейме из словаря 'all_raw_data'
all_raw_data = {raw_data[0]: raw_data[1].rename(columns=raw_data[1].iloc[0])[2:]
                                        .reset_index(drop=True) for raw_data in all_raw_data.items()}

In [ ]:
# удалим строку с итогами
all_raw_data = {raw_data[0]: raw_data[1].iloc[:-1] for raw_data in all_raw_data.items()}

In [ ]:
# переименуем столбцы
for row_key, row_value in all_raw_data.items():
    row_value.columns = ['Субконто 1',
                         'Счёт',
                         'Субконто 2',
                         'Сумма нач. ост. Дт',
                         'Сумма нач. ост. Кт',
                         'Сумма оборотов Дт',
                         'Сумма оборотов Кт',
                         'Сумма кон. ост. Дт',
                         'Сумма кон. ост. Кт']

In [ ]:
# при помощи цикла поменяем местами столбцы
all_raw_data = {raw_data[0]: raw_data[1].iloc[:, [0, 2, 1, 3, 4, 5, 6, 7, 8]] for raw_data in all_raw_data.items()}

In [ ]:
for row_key, row_value in all_raw_data.items():
    if not row_value.empty:
        # найдём первое вхождение уникального значения в столбце 'Счёт'
        # и создадим словарь со значением по столбцу 'Счёт' и индексу этого значения
        unique_vals = {val: i for i, val in reversed(list(enumerate(x for x in row_value['Счёт'])))}
        del unique_vals[np.nan]
        unique_vals_list_1 = [value for value in unique_vals.values()]
        row_value['Является итогом по счёту'] = row_value.index.values
        row_value['Является итогом по счёту'] = np.where(row_value['Является итогом по счёту'].isin(unique_vals_list_1),
                                                         'Да',
                                                         'Нет')
    
        na_values = list(row_value['Субконто 1'][lambda x: pd.isnull].index)
        unique_vals_list_2 = na_values + unique_vals_list_1
        row_value['Является итогом по Субконто 1'] = row_value.index.values
        row_value['Является итогом по Субконто 1'] = np.where(~(row_value['Является итогом по Субконто 1'].isin(unique_vals_list_2)),
                                                              'Да',
                                                              'Нет')
    else:
        row_value['Является итогом по счёту'] = np.nan
        row_value['Является итогом по Субконто 1'] = np.nan

In [ ]:
# заполним значения в столбцах 'Счёт' и 'Субконто 1' вниз
for row_key, row_value in all_raw_data.items():
    row_value['Счёт'] = row_value['Счёт'].fillna(method='ffill',
                                                 axis=0)
    row_value['Субконто 1'] = row_value['Субконто 1'].fillna(method='ffill',
                                                             axis=0)

In [ ]:
# отфильтруем датафреймы
all_raw_data = {raw_data[0]: raw_data[1][(raw_data[1]['Является итогом по счёту'] == 'Нет') & 
                                         (raw_data[1]['Является итогом по Субконто 1'] == 'Нет')]
                for raw_data in all_raw_data.items()}

In [ ]:
# удалим лишние столбцы
all_raw_data = {raw_data[0]: raw_data[1].drop(['Является итогом по счёту',
                                               'Является итогом по Субконто 1'],
                                              axis=1)
                for raw_data in all_raw_data.items()}

In [ ]:
# создадим списки из кортежей с названием файла и началом/концом периода
raw_data_period_start = list(zip(raw_data_name_list, period_list_start))
raw_data_period_end = list(zip(raw_data_name_list, period_list_end))

In [ ]:
# при помощи цикла добавим столбец 'Начало периода' в каждый датафрейм из словаря 'all_raw_data'
for raw_data in all_raw_data.items():
    for period in raw_data_period_start:
        if raw_data[0] == period[0]:
            raw_data[1]['Начало_периода'] = period[1]

# при помощи цикла добавим столбец 'Конец периода' в каждый датафрейм из словаря 'all_raw_data'
for raw_data in all_raw_data.items():
    for period in raw_data_period_end:
        if raw_data[0] == period[0]:
            raw_data[1]['Конец_периода'] = period[1]

In [ ]:
# сбросим индексы
for row_key, row_value in all_raw_data.items():
    row_value.reset_index(inplace=True, drop=True)

In [ ]:
# словарь с именами столбцов и типами данных
type_of_columns = {'Сумма нач. ост. Дт': 'float64', 
                   'Сумма нач. ост. Кт': 'float64', 
                   'Сумма оборотов Дт': 'float64',
                   'Сумма оборотов Кт': 'float64', 
                   'Сумма кон. ост. Дт': 'float64', 
                   'Сумма кон. ост. Кт': 'float64'}

In [ ]:
# изменим тип данных в каждом датафрейме в соответствии со словарём 'type_of_columns'
all_raw_data = {raw_data[0]: raw_data[1].astype(type_of_columns) for raw_data in all_raw_data.items()}

In [ ]:
# изменим тип данных в столбцах 'Начало периода' и 'Конец периода' в каждом датафрейме
for row_key, row_value in all_raw_data.items():
    row_value['Начало_периода'] = row_value['Начало_периода'].apply(lambda row: pd.to_datetime(row, format='%d.%m.%Y'))
    row_value['Конец_периода'] = row_value['Конец_периода'].apply(lambda row: pd.to_datetime(row, format='%d.%m.%Y'))

In [ ]:
'''
Цикл, который перезаписывает каждый файл из 'all_raw_data', чтобы сохранить внесённые ранее изменения в его структуру,
а также сохраняет файл в формате csv в первоначальную папку
'''
print(f'Перезапись ОСВ из "{source}"')
for raw_data in tqdm(all_raw_data.items()):
    raw_data[1].to_csv(os.path.join(source, raw_data[0].replace('.xlsx', '.csv').replace('(XLSX)', '(CSV)')),
                       header=True,
                       index=False)

In [ ]:
# создадим список из кортежей с названием файла и периодом
raw_data_period = list(zip(raw_data_name_list, period_list_end_reversed))

In [ ]:
# создадим список с расположением новых папок
folder_location = [os.path.join(source,
                                (raw_data[0].split('_')[0]
                                            .replace('предыдущий месяц',
                                                     'На конец месяца')),
                                f'{raw_data[1]}_ОСВ')
                   for raw_data in raw_data_period]
# цикл создаёт папки, в которые нужно переместить исходные данные
for location in folder_location:
    # создадим папки только для данных на конец месяца
    if 'На конец месяца' in location:
        # создаются только те папки, которые до этого не существовали
        if not os.path.exists(location):
            os.makedirs(location)

In [ ]:
# объединим списки 'raw_data_name_list' и 'folder_location'
# в список кортежей с названием файла и расположением новой папки для данного файла
raw_data_list = list(zip(raw_data_name_list, folder_location))

In [ ]:
# цикл, который копирует каждый файл из 'raw_data_list' в новую папку и удаляет его из папки c исходными данными
print(f'Распределение ОСВ из "{source}" по папкам')
for raw_data in tqdm(raw_data_list):
    if 'предыдущий месяц' in raw_data[0]:
        # скопируем исходные данные с расширением .csv в новую папку
        shutil.copy(os.path.join(source,
                                 raw_data[0].replace('.xlsx', '.csv').replace('(XLSX)', '(CSV)')),
                    os.path.join(raw_data[1],
                                 '_'.join(raw_data[0].replace('.xlsx', '.csv').replace('(XLSX)', '(CSV)').split('_')[1:])))
    # удалим исходные данные с расширением .csv из первоначальной папки
    os.remove(os.path.join(source, raw_data[0].replace('.xlsx', '.csv').replace('(XLSX)', '(CSV)')))
    # удалим исходные данные с расширением .xlsx из первоначальной папки
    os.remove(os.path.join(source, raw_data[0]))

## Преобразование данных для выгрузки в базу данных

In [ ]:
# удалим лишние столбцы в каждой таблице
all_raw_data = {raw_data[0]: raw_data[1].drop(['Сумма нач. ост. Дт',
                                               'Сумма кон. ост. Дт',
                                               'Сумма оборотов Дт',
                                               'Сумма оборотов Кт'],
                                              axis='columns')
                for raw_data in all_raw_data.items()}

In [ ]:
# переименуем столбцы в каждой таблице
all_raw_data = {raw_data[0]: raw_data[1].rename(columns={'Сумма нач. ост. Кт': 'Задолженность_на_начало_периода',
                                                         'Сумма кон. ост. Кт': 'Задолженность_на_конец_периода',
                                                         'Субконто 1': 'Кредитор',
                                                         'Субконто 2': 'Договор',
                                                         'Счёт': 'Тип_задолженности'},
                                                inplace=False)
                for raw_data in all_raw_data.items()}

In [ ]:
# добавим столбец 'Заёмщик' в каждый датафрейм
for key, value in all_raw_data.items():
    value['Заёмщик'] = key.split('_')[-1].split(' (')[0]

In [ ]:
# добавим столбец 'Номер договора' в каждый датафрейм
for key, value in all_raw_data.items():
    value['Номер_договора'] = value['Договор'].apply(lambda row: row.split(' от')[0])

In [ ]:
# добавим столбец 'Дата заключения договора' в каждый датафрейм
for key, value in all_raw_data.items():
    value['Дата_заключения_договора'] = value['Договор'].apply(lambda row: 
                                                               re.search(r'\d{2}\.\d{2}\.\d{4}', # шаблон для поиска даты
                                                                         row.split('от ')[1]).group(0))

In [ ]:
# изменим тип данных в столбце 'Дата заключения договора' в каждом датафрейме
for row_key, row_value in all_raw_data.items():
    row_value['Дата_заключения_договора'] = row_value['Дата_заключения_договора'].apply(lambda row:
                                                                                        pd.to_datetime(row, format='%d.%m.%Y'))

In [ ]:
# заменим все 'NaN' на 0 в столбцах 'Задолженность на начало периода' и 'Задолженность на конец периода'
for key, value in all_raw_data.items():
    value['Задолженность_на_начало_периода'].replace(np.nan, 0, inplace=True)
    value['Задолженность_на_конец_периода'].replace(np.nan, 0, inplace=True)

In [ ]:
# добавим столбец 'Изменение задолженности' в каждый датафрейм
for key, value in all_raw_data.items():
    value['Изменение_задолженности'] = value['Задолженность_на_конец_периода'] - value['Задолженность_на_начало_периода']

In [ ]:
# словарь с типом задолженности
type_of_debt = {'66.01': 'Краткосрочный кредит',
                '66.02': 'Проценты по краткосрочному кредиту',
                '66.03': 'Краткосрочный займ',
                '66.04': 'Проценты по краткосрочному займу',
                '66.05': 'Краткосрочные долговые ценные бумаги',
                '66.06': 'Проценты по краткосрочным долговым ценным бумагам',
                '67.01': 'Долгосрочный кредит',
                '67.02': 'Проценты по долгосрочному кредиту',
                '67.03': 'Долгосрочный займ',
                '67.04': 'Проценты по долгосрочному займу',
                '67.05': 'Долгосрочные долговые ценные бумаги',
                '67.06': 'Проценты по долгосрочным долговым ценным бумагам'}

In [ ]:
# заменим значения в столбце 'Тип задолженности' на значения в словаре 'type_of_debt'
for key, value in all_raw_data.items():
    value['Тип_задолженности'] = value['Тип_задолженности'].replace(type_of_debt)

In [ ]:
# добавим столбец 'Уникальный ключ' в каждый датафрейм
for key, value in all_raw_data.items():
    value['Уникальный_ключ'] = value['Конец_периода'].apply(lambda row: str(row)) + value['Номер_договора'].apply(lambda row: row)

In [ ]:
# создадим список для хранения таблиц с данными за предыдущий месяц и текущий день
tables_to_merge_last_month = [] 
tables_to_merge_current_day = [] 
# цикл для заполнения списка определёнными таблицами
for key, value in all_raw_data.items():
    if key.startswith('предыдущий месяц'):
        tables_to_merge_last_month.append(value)
        
for key, value in all_raw_data.items():
    if key.startswith('текущий день'):
        tables_to_merge_current_day.append(value)        

In [ ]:
# объединим таблицы
merged_osv_last_month = pd.concat(tables_to_merge_last_month, axis=0).reset_index(inplace=False, drop=True)
merged_osv_current_day = pd.concat(tables_to_merge_current_day, axis=0).reset_index(inplace=False, drop=True)

In [ ]:
# сортированный список столбцов
sort_columns = ['Уникальный_ключ',
                'Начало_периода',
                'Конец_периода',
                'Заёмщик',
                'Кредитор',
                'Тип_задолженности',
                'Договор',
                'Номер_договора',
                'Дата_заключения_договора',
                'Задолженность_на_начало_периода',
                'Задолженность_на_конец_периода',
                'Изменение_задолженности']

In [ ]:
# поменяем местами столбцы
merged_osv_last_month = merged_osv_last_month.reindex(columns=sort_columns)
merged_osv_current_day = merged_osv_current_day.reindex(columns=sort_columns)

In [ ]:
# создадим отдельные таблицы с процентами по задолженности
merged_osv_last_month_procent = (merged_osv_last_month[merged_osv_last_month['Тип_задолженности'].str.contains('Проценты')]
                                 .reset_index(drop=True))
merged_osv_current_day_procent = (merged_osv_current_day[merged_osv_current_day['Тип_задолженности'].str.contains('Проценты')]
                                  .reset_index(drop=True))

In [ ]:
# отфильтруем таблицы, удалив строки с процентами по задолженности
merged_osv_last_month = (merged_osv_last_month[~merged_osv_last_month['Тип_задолженности'].str.contains('Проценты')]
                         .reset_index(drop=True))
merged_osv_current_day = (merged_osv_current_day[~merged_osv_current_day['Тип_задолженности'].str.contains('Проценты')]
                          .reset_index(drop=True))

## Подключение к PostgreSQL и выгрузка csv-файлов в базу данных

In [ ]:
# создадим справочник с данными для подключения к БД
db_config = {
        'user': 'user',
        'password': 'password',
        'host': 'host',
        'port': 'port',
        'database': 'database'
    }

In [ ]:
# создадим строку для подключения к БД
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
        db_config['user'],
        db_config['password'],
        db_config['host'],
        db_config['port'],
        db_config['database']
)

In [ ]:
# подключимся к БД через sqlalchemy
engine_alc = db.create_engine(connection_string)
conn_alc = engine_alc.connect()

In [ ]:
# подключимся к БД через psycopg2
conn_psy = psycopg2.connect(database=db_config['database'],
                            user=db_config['user'],
                            password=db_config['password'],
                            host=db_config['host'], 
                            port=db_config['port'])

In [ ]:
# выгрузим объединённые таблицы с данными по предыдущему месяцу в БД
merged_osv_last_month.to_sql('debt_month', engine_alc, schema='schema', if_exists='append', index=False)
merged_osv_last_month_procent.to_sql('debt_month_procent', engine_alc, schema='schema', if_exists='append', index=False)

In [ ]:
# выгрузим объединённые таблицы с данными по текущему дню в БД
merged_osv_current_day.to_sql('debt_day', engine_alc, schema='schema', if_exists='append', index=False)
merged_osv_current_day_procent.to_sql('debt_day_procent', engine_alc, schema='schema', if_exists='append', index=False)

In [ ]:
# функция для удаления дубликатов из таблицы в БД (остаются только последние добавленные уникальные договоры)
def del_duplicates_month(table_name, schema):
    cur.execute(f'DELETE \
                  FROM {schema}.{table_name} \
                  WHERE Уникальный_ключ IN (SELECT Уникальный_ключ \
                                            FROM {schema}.{table_name} \
                                            GROUP BY Уникальный_ключ \
                                            HAVING COUNT(*) > 1) AND Обновлено NOT IN (SELECT MAX(Обновлено) \
                                                                                       FROM {schema}.{table_name} \
                                                                                       GROUP BY Уникальный_ключ \
                                                                                       HAVING COUNT(*) > 1);')

In [ ]:
# функция для удаления дубликатов из таблицы в БД (остаются только последние добавленные уникальные договоры)
def del_duplicates_day(table_name, schema):
    cur.execute(f'DELETE \
                  FROM {schema}.{table_name} \
                  WHERE Договор IN (SELECT Договор \
                                           FROM {schema}.{table_name} \
                                           GROUP BY Договор \
                                           HAVING COUNT(*) > 1) AND Обновлено NOT IN (SELECT MAX(Обновлено) \
                                                                                      FROM {schema}.{table_name} \
                                                                                      GROUP BY Договор \
                                                                                      HAVING COUNT(*) > 1);')

In [ ]:
# cоздадим переменную cur и присвоим ей объект курсора, созданный для подключения к базе данных conn_psy
# курсор используется для выполнения запросов SQL и получения результатов из базы данных
cur = conn_psy.cursor()

# применим функцию del_duplicates
del_duplicates_month('debt_month', 'schema')
del_duplicates_month('debt_month_procent', 'schema')
del_duplicates_day('debt_day', 'schema')
del_duplicates_day('debt_day_procent', 'schema')

In [ ]:
# сохраним изменения и закроем соединение с БД через psycopg2
conn_psy.commit()
cur.close()
conn_psy.close()